# Text classification with the Naive Bayes algorithm #

**Machine Learning** (ML) is a sub-field of Artificial Intelligence (AI) that automates analytical model building. As the amount of data grows, ML is becoming the defacto standard within all research fields. So what does it mean that a machine *learns*? Given enough data, we can offload programming tasks to the algorithm, that is, the algorithm learns structure from the data:

Tom Mitchell's **A well-posed learning problem**: "A computer program is said to learn from experience $E$ with respect to some task $T$ and some performance measure $P$, if its performance on $T$, as measured by $P$, improves with experience $E$"

**Classification** is one is a primary task of supervised learning. Given labeled data, a classification algorithm will output a solution that categorizes new examples (i.e., associate labels with subsets of the data). While unsupervised learning searches for groups within the data, classification learns to map a data set onto a categorical class values or labels (i.e., function approximation).

The **Naive Bayes** (NB) algorithm is a generative algorithm that is very popular for text classification. 
The probability of a document $d$ being in class $c$, $P(c \mid d)$ is computed as:


$$ P(c \mid d) \propto P(c) \prod_{i = 1}^{m}P(t_i \mid c) $$
and the class of a document $d$ is then computed as:

$$c_{MAP} = arg~max_{c \in \{c_1, c_2 \}} P(c \mid d)$$


## Research problem ##

The medieval writer Saxo Grammatricus (c. 1160 - post 1208) represents the beginning of the modern day historian in Scandinavia. Saxo's history of the Danes *Gesta Danorum* ("Deeds of the Danes") is the single most important written source to Danish history in the 12<sup>th</sup> century. *Gesta Danorum*  is tendentious, contains elements of fiction, and its compositions has been an academic subject of debate for more than a century. The more recent debate treat the bipartite composition *Gesta Danorum* and centers on two related issues: 1) is the transition between the old mythical and new historical part located in book eight, nine, or ten; and 2) is this transition gradual (continuous) or sudden (point-like)? In this tutorial we will ask "is book nine more similar to the early books (1-8) or the late books (10-16)". This is an example uses simple vector space techniques from author chronometry to represent the most salient stylistic and semantic leaxical features of *Gesta Danorum*.

In [8]:
import os
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from collections import Counter


data_path = os.path.join("data","saxo_books","saxo_class.csv")

data = pd.read_csv(data_path)

#print(data.tail())

print(set(data["book_class"]))

{'late', 'uncertain', 'early'}


### split data ###

In [9]:
idx = data["book_class"] == "uncertain"

data_uncertain = data[idx]
data = data[~idx]

In [14]:
def print_dist(df):
    """ print distribution of classes
    """
    for label in set(df["book_class"]):
        print("number of {}: {}".format(label,sum(df["book_class"] == label)))

print_dist(data)

# Ross' pandas magic
data["book_class"].describe()

number of late: 707
number of early: 518


count     1225
unique       2
top       late
freq       707
Name: book_class, dtype: object

In [18]:
ratio = 0.8
mask = np.random.rand(len(data)) <= ratio

train = data[mask]
test = data[~mask]

## training set
X_train = train["content"].values
y_train = train["book_class"].values

# testing
X_test = test["content"].values
y_test = test["book_class"].values

In [21]:
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words = None, lowercase = True, 
                             max_df = 0.9, min_df = 0.01, max_features = 500)

feature_train = vectorizer.fit_transform(X_train)
feature_test = vectorizer.fit_transform(X_test)
feature_names = vectorizer.get_feature_names()

print(feature_names)

['absalon', 'ad', 'af at', 'af de', 'af dem', 'af den', 'af en', 'af hans', 'af med', 'af sider', 'af sin', 'af sine', 'af sted', 'al', 'aldrig', 'alene', 'alle', 'allerede', 'alt', 'altid', 'altså', 'anden', 'andet', 'andre', 'angreb', 'at blive', 'at de', 'at den', 'at der', 'at det', 'at få', 'at give', 'at gøre', 'at han', 'at have', 'at holde', 'at komme', 'at man', 'at se', 'at sætte', 'at tage', 'bad', 'bag', 'bare', 'bedre', 'begge', 'begyndte', 'besked', 'besluttede', 'blandt', 'blev', 'blev han', 'blevet', 'blive', 'bror', 'byen', 'både', 'da', 'da de', 'da det', 'da han', 'dag', 'danmark', 'danmarks', 'danmarks historie', 'danske', 'danskerne', 'de havde', 'de ikke', 'de to', 'de var', 'del', 'dem', 'dem der', 'dem til', 'den anden', 'den der', 'den ene', 'den mand', 'den måde', 'denne', 'der havde', 'der ikke', 'der skulle', 'der var', 'deres', 'derfor', 'derpå', 'det er', 'det gik', 'det ikke', 'det var', 'dette', 'dig', 'din', 'disse', 'dog', 'drog', 'du', 'død', 'døden',

In [28]:
nb_classifier = MultinomialNB()
nb_classifier.fit(feature_train,y_train)

# evaluate
pred = nb_classifier.predict(feature_test)

# accuracy
print("Accuracy: {}".format(round(metrics.accuracy_score(y_test,pred),2)))

print(707/1225)

Accuracy: 0.62
0.5771428571428572


In [30]:
feature_uncertain = vectorizer.transform(data_uncertain["content"])
uncertain_class = nb_classifier.predict(feature_uncertain)
decision= Counter(uncertain_class)

print("book 9 has {} votes for early style and {} for late style".format(decision["early"],decision["late"]))

book 9 has 16 votes for early style and 27 for late style
